<a href="https://colab.research.google.com/github/npd2013/Mypublicmodels/blob/main/Experiment_7_4_vectorspace_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SETUP1:-  
-Single document text summarization 

-frequency based method for scoring sentences 

-preprocessing applied are knowledge based normalization,  stopword removal and stemming.
-Using  CUE WORDS - actors in story.

-automatic evaluation using rouge and compression ratio analysis.

note:results stored in another file 


Algorithm steps :- 

1) Preprocessing of text- 

    1.1)Cleaning 

    1.1.1)Removal of unnecessary symbols/charachters like html tags ,extra spaces,extra newlines etc.
         
 
 1.2) Line segmentation
 
    -using indicnlp with own segmenter  
    -Divide text into lines and find token words using own word segmenter.
 1.3) word segmentation
    -Divide the lines into words.
    -words can be separated by whitespace or characters like quotes or comma.  use the punctuation list to exclude from the word.Only extract valid words 

  1.4) Stopword removal 
 
       -basic set of 15 words removed from lines. 
   
 1.5) Stemming
    -if word is multiword expression,return  

    -make dictionary of valid words using indowordnet 
 
    -remove salutations from suffix like bhai,bahen etc
 
    -Correct wordspelling as per dictionary, if valid dictionary word. 

    -if not valid dictionary word

       -look up prefix and trim if match found

       -look up replacement and trim if match found

       - look up longest suffix rules matching and trim if match found.
    
    -if trimmed word size>=3 
            
            -    stemword = trimword
   
                   
               
       

2)Make vector space representation -based on term frequency

    2.1) Compute tfidf using sklearn module of python.

    2.2) For each sentence - 

     2.2.1) Compute sentence score= total tfidf score of all words in sentence divide by number of words in sentence.
     2.2.2) If it is first or last sentence , increase its weightage ten times.
  
  2.3) For each sentence -

     3.2.1) Compute the average normalized score  : 
          Current sentence score divide by total sentence score of text.
      

 4) //Find top "n" scored sentences.

 4.1) Set n= number of lines in gold summary. 0r "x" percent of original text.    
 4.2) Summary_length=n
 4.3) Sorted_sentences=sort(sentences) based on their scores.
    
 5) //Generate the summary -

    for sentences = 1 to gold_summary_length 
      - if sentence in gold_summary  
           put sentence in system_summary. 
  
6)// Evaluate the summary-

    6.1) use the Rouge score 
      - Get the precision ,recall and F-measure for rouge-1 ,rouge2 and rouge-LCS.   

    6.2) Find comression ratio of text
       6.2.1) sentence compression ratio
        = total sentences in system_summary/total sentences in source text 
        6.2.2) Word compression ratio
        =total words in system_summary/total words in source 
  


In [1]:
#install needed packages 
#!pip install  gensim  #has word2vec model built, using english 

!pip install pyiwn
!pip install -U scikit-learn
!pip install rouge
!pip install indic-nlp-library


     |████████████████████████████████| 22.3MB 6.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
from google.colab import files #upload the files from local machine on the colab environment 
uploaded = files.upload()


Saving story1.txt to story1.txt
Saving story1-gold.txt to story1-gold.txt
Saving story1-topic.txt to story1-topic.txt
Saving story2.txt to story2.txt
Saving story2-gold.txt to story2-gold.txt
Saving story2-topic.txt to story2-topic.txt
Saving story3.txt to story3.txt
Saving story3-gold.txt to story3-gold.txt
Saving story3-topic.txt to story3-topic.txt
Saving story4.txt to story4.txt
Saving story4-gold.txt to story4-gold.txt
Saving story4-topic.txt to story4-topic.txt
Saving story5.txt to story5.txt
Saving story5-gold.txt to story5-gold.txt
Saving story5-topic.txt to story5-topic.txt
Saving story6.txt to story6.txt
Saving story6-gold.txt to story6-gold.txt
Saving story6-topic.txt to story6-topic.txt
Saving story7.txt to story7.txt
Saving story7-gold.txt to story7-gold.txt
Saving story7-topic.txt to story7-topic.txt
Saving story8.txt to story8.txt
Saving story8-gold.txt to story8-gold.txt
Saving story8-topic.txt to story8-topic.txt
Saving story9.txt to story9.txt
Saving story9-gold.txt t

In [3]:
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_preporcess-v6.py" "guj_final_preporcess.py" #The preprocessing modules designed
!cp "/content/drive/My Drive/Colab Notebooks/Evaluation.py" "Evaluation.py"   # the evaluation module designed
!cp "/content/drive/My Drive/Colab Notebooks/guj_final_postprocessing-v1.py" "post.py"   # Get the post processing module 
!cp "/content/drive/My Drive/Colab Notebooks/my-dictionary-gujarati.csv" "my-dictionary-gujarati.csv" #the static dictionary of words in gujarati


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import io
import csv
from Evaluation import *
from post import *
import pandas as pd 
from guj_final_preporcess import * #to get the required preporcessing modules -from self made pipeline
import pyiwn

2021-03-13:16:36:01,13 INFO     [helpers.py:20] Downloading IndoWordNet data of size ~31 MB...


[██████████████████████████████████████████████████]

2021-03-13:16:36:04,859 INFO     [helpers.py:43] Extracting /root/iwn_data.tar.gz into /root...


2021-03-13:16:36:06,695 INFO     [helpers.py:48] Removing temporary zip file from /root/iwn_data.tar.gz
2021-03-13:16:36:06,702 INFO     [helpers.py:51] IndoWordNet data successfully downloaded at /root/iwn_data


In [303]:
#take all the needed inputs from user 
basename=input('Enter the basic name of file(without txt),other names wud be autogenerated  ')

filename=basename+'.txt'
gold_filename=basename+'-gold.txt'
#filename=input('Enter text file name to be summarized ') #Ex: file1.txt
analysefile_name=filename.replace('.txt','analysis-TF--without-stemtolemma.txt')# to store the data for later analysis . Example- file1-results.txt 
features_filename=filename.replace('.txt','TF-Features-without-stemtolemma.csv') # file-1-features.csv
filename_statistics='datastatistics-TF--without-stemtolemma.csv'
#gold_filename=input("Enter name of file containing gold summary  ") #example file1-gold.txt ,assumes the name is having pattern ending with gold.txt 
summary_filename=filename.replace('gold.txt','-TF-systemsummary-without-stemtolemma.txt') # example file1-gold-system-summary.txt
topic_filename=basename+'-topic.txt'

rougescorefile="Rougeresults-TF-without-stemtolemma.csv"
dictionaryfile="my-dictionary-gujarati.csv"

Enter the basic name of file(without txt),other names wud be autogenerated  story40


In [304]:
"""print("enter modules you want to activate in current run ")
vectormodel=input("1- Using TFIDF technique  2- Using TF technique :: your choice")
stopwordflag=input("3-With stopword removal phase    4- Without stopword removal phase:: your choice")
stemmerflag=input("5- With stemmer phase     6- Without stemmer phase:: your choice ")
lemmaflag=input("7- With stem to lemma c3onversion.  8- Without stem to lemma conversion:: your choice")
"""
vectormodel=2
stopwordflag=3
stemmerflag=5
lemmaflag=8

In [305]:
#read the input original file . 

guj_clean_html(filename,filename)# text cleaning remove the html tags and overwrite the original file 
guj_clean_html(gold_filename,gold_filename)# clean the html tags from gold file 
guj_clean_html(topic_filename,topic_filename)

guj_clean_extra(filename,filename)#remove the extra symbols like multiple punctuations and overwrite original file
guj_clean_extra(gold_filename,gold_filename)
guj_clean_extra(topic_filename,topic_filename) 
#Display the given data in input files 
with io.open(filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file
analyzedfile_obj=open(analysefile_name,mode='w',encoding='UTF-8')
analyzedfile_obj.write("\n\n The original file is :------\n")
analyzedfile_obj.write(lines)

with io.open(gold_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The gold summary provided by expert  is -->\n")    
analyzedfile_obj.write(lines)
with io.open(topic_filename,'r',encoding='utf-8') as f: 
		  lines=f.read()#get all the lines from file

analyzedfile_obj.write("\n\n The topic source file given is :------\n")
analyzedfile_obj.write(lines)



# now find the statistics of originally given informations 
originalcorpora,totaloriginalsentences=guj_corpus_generate(filename) #make corpus of original data 
goldcorpora,totalgoldcorpora=guj_corpus_generate(gold_filename) #make corpus of gold 

analyzedfile_obj.write("\n\n STATISTICS------>")
analyzedfile_obj.writelines("\n Original file sentences(RAW) - {0}\n".format(totaloriginalsentences))
analyzedfile_obj.writelines("\n gold file sentences(RAW)- {0}\n".format(totalgoldcorpora))

###################################################################
########################  Step-1 ##################################
#####################  cleaning phase  ########################

#doing cleaning of file 
#remove newlines and other unnecessary data and clean it for feature extraction 
#to replace the data as per our format, replacing abbreviations dot with '*'
normfile="temp1.txt"
norm_gold_file="temp2.txt"
clean_topic_file=norm_topic_file="temp3.txt"

guj_clean_text(filename,normfile) 

guj_clean_text(gold_filename,norm_gold_file)
guj_clean_text(topic_filename,clean_topic_file) #no further process needed for topic file, it is already cleaned.

#Now display the stats and make corpora after cleaning file and removing new lines . 
normcorpora,totalnormsentences=guj_corpus_generate(normfile)  
goldcorpora,totalnormgoldsentences=guj_corpus_generate(norm_gold_file) 
topiccorpora,totaltopicsentences=guj_corpus_generate(norm_topic_file)

analyzedfile_obj.write(" Segmented Lines in source -{0} \n ".format(totalnormsentences))
analyzedfile_obj.write(" Segmented Lines in gold -{0}\n ".format(totalnormgoldsentences))
analyzedfile_obj.write(" the total sentences in normalized topic file are= {0} ".format(totaltopicsentences))
###################################################################
########################  Step-2 ##################################
#####################  Normalization and tokenization  for feature extraction  ########################

cleanfile="file.txt"
stemmedlist=[] #to store the stemmed words 
lemmalist=[] #to store the lemmas(dictionary) words found from vacabulory 
mydictionary=guj_makedictionary(dictionaryfile) #generate the dictionary of unique words from given file of gujarati words

if (stopwordflag=='3'):
 cleanfile=guj_stopwordremoval(normfile,cleanfile) #remove stopwords done 
else :
  cleanfile=normfile # no stopword removal step done ,so clean file is directly the normalized one 

scorpora,stotalsent=guj_corpus_generate(cleanfile) #scorpora- the changed lines after stopword removal 

focus_list=guj_makedictionary(topic_filename)

if (stemmerflag=='5'):
  cleanfile,stemmedlist,lemmalist=guj_stemmer_new(cleanfile,cleanfile,mydictionary,focus_list) #stem with help from dictionary ,and get back two lists 

stemcorpora,stemtotalsent=guj_corpus_generate(cleanfile) #stemcorpora- the changed lines after stemming 

if (lemmaflag=='7'):
  cleanfile=guj_stem_to_lemma(cleanfile,stemmedlist,lemmalist) #if have both stem and dictionary form of word ,convert to the dictionay form 


vocab=guj_makevocab(cleanfile)

#!!!!!!!!!!!!!!!!!!!For finding the actors / Focus group in story !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#####

vocab_topic=guj_makevocab(clean_topic_file) #extra words to be included in the feature extraction ,based on topic

#Now working on the modified file to get various features 

corpora,totalsent=guj_corpus_generate(cleanfile) #divide lines into sentences-
analyzedfile_obj.write("\n Lines in source after all processing -{0} \n ".format(totalsent))

sent_pos_list=[] #to store the sentence position
sent_words_count=[]  # to store the total words in each sentence 
sent_number=0
for s in corpora:
  sent_pos_list.append(sent_number)
  sent_number+=1

  

# to generate vector represnetations  
if (vectormodel=='1'):
 vectorizer = TfidfVectorizer(tokenizer = guj_mytokenizer,min_df=0.15,max_df=0.85) #frequency of words with atleast 15% frequency
else: 
 vectorizer = CountVectorizer(tokenizer = guj_mytokenizer,min_df=0.15,max_df=0.85) #frequency of words with atleast 15% frequency

cwm = vectorizer.fit_transform(corpora) #vectorize the corpus of input file


tokens = vectorizer.get_feature_names() #get the names of words used in vector
#print("features are - ",tokens )
#### Extra logic for topic based vectorizor feature upadation ###########333
n=0
top_features = vectorizer.vocabulary_ #to store the features extracted
for f in top_features.keys():
  n+=1 #top features total count

analyzedfile_obj.write("\ntop features and their values were{0} -".format(top_features))
pos=n
for feature, index in vocab_topic.items(): #take all topic words from vocab and add in features 
    
    if feature not in top_features.keys():#add only if not already considered in top words 
          top_features[feature] = pos
          pos=pos+1

 
# re-vectorize using both sets of features
if (vectormodel=='1'):
 vectorizer = TfidfVectorizer(vocabulary=top_features,tokenizer = guj_mytokenizer)
else :
 vectorizer = CountVectorizer(vocabulary=top_features,tokenizer = guj_mytokenizer)   

cwm = vectorizer.fit_transform(corpora)
tokens = vectorizer.get_feature_names() #get the names of words used in vector

top_features = vectorizer.vocabulary_
analyzedfile_obj.write("\n New top features after considering the actor words are {0} -".format(top_features))
#### End of extra logic for adding separate topic based keywords in learning 
score_list=[]
counts=cwm.sum(axis=1) #sums the sentence wise words' tfidf scores
scorelist=list(counts.flat)#get the list of sentencescores 


analyzedfile_obj.write("\n\n****The TF  based vector is formed ****")
analyzedfile_obj.write('\n The dimension of vector matrix generated is -\n {0} '.format(cwm.shape))

##########################################################
################# ######Step3#############################
################ Compute sentence scores #################
##########################################################

totwords=[]

for s in corpora:
   count=0
   words=guj_mytokenizer(s)
   for c in words:
    count=count+1
   totwords.append(count)


list_of_tuples = list(zip(sent_pos_list,normcorpora,scorpora,stemcorpora,corpora,originalcorpora ,cwm.toarray(),scorelist,totwords))  
df=pd.DataFrame(list_of_tuples, columns=['sent_pos','preprocessed_line','stopword_removed','stemmed_line','sentence_final','raw sentence','feature_vector','totalsentcount','totalwordsinsent'])

df['sent_score']=df['totalsentcount']/df['totalwordsinsent']
#based on sentence position , increase weightage of first and lasat sentences in txt 
df.at[0,'sent_score']=df.at[0,'sent_score']*10 
try:
  df.at[sent_number-1,'sent_score']=df.at[sent_number-1,'sent_score']*10 
except:
   print()


df.to_csv(features_filename) 




In [306]:
 rougescorefilehandle=open(rougescorefile,mode='a',encoding="utf-8") #to store only the rouge scores for later analysis 
 resultvar = csv.writer(rougescorefilehandle, lineterminator='\n')		#to store new row for each f  


################# ######Step4#############################
################ Find top sentences  #################

#rank and sort the sentences  based on individual sentences score 

 percent=(totalnormgoldsentences/totalnormsentences)*100 #ideal ratio as per gold text given 
 n=totalnormgoldsentences#extract sentences as per total sentences in gold summary 
 data=df.sort_values(["sent_score"],ascending=False,kind='quicksort')
 resultlist=[]#to store the rouge score data

 resultlist.append(filename+",") #mention the source file name for record
 
 top_sent=data.iloc[0:n,1:2]  #get the sentences in top 
 # print(top_sent.head())
 toplist=list(top_sent.preprocessed_line)
 #print("top sentences are ",toplist)
 analyzedfile_obj.write("\n\n List of top sentences are as follows-{0} ".format(toplist))
  ##########################################################
################# ######Step6#############################
################ Generate summary  #################
##########################################################
 summary=[]
 not_in_summary=[]
 sentence_number=0
 summary_lines=[]
 for sent in normcorpora:  #take one by one the sentences considered in feature vector 
    #print("sentence in norm corpora is ",sent)
    if sent in toplist: #if sentence is in topranked sentences then store its number so its original line can be fetched from original corpus
        summary.append(sentence_number)
        summary_lines.append(sent)
        #print("sentence included in summary is ",sent)
    else:
        not_in_summary.append(sentence_number)

    sentence_number=sentence_number+1

 analyzedfile_obj.write("\n\n ***** The summary geneated has the following numbered sentences ---->>\n {0}".format(summary))
 analyzedfile_obj.write("\n\n ***** The sentences not included in summary are these numbers---->>{0}".format(not_in_summary))
  
 
  #store the summary in file
 summary_filename='system-temp-{0}.txt'.format(str(percent))
    #to give name tosummary file for storing percent based results
  #format is  original file name -system-<percent> Ex:- news1-system-50
  
 with io.open(summary_filename,'w',encoding='utf-8') as w: 
    for i in summary_lines:
      w.write(i)
      
      
  
  ###########step5#############
  ########### Post Processing ######
 resultfilename="system_summary_TF+stopwordr+stem_"+filename
 #post_process(summary_filename,summary_filename)#do the post processing and write in result file, remove adj and adv from final file
 
 post_process_witout_wordnet(summary_filename,summary_filename) #doint post processing without using wordnet to remove adj and adverbs
 with io.open(summary_filename,'r',encoding='utf-8') as r: 
         lines=r.read()
 corp,count=guj_corpus_generate(summary_filename)      
 with io.open(resultfilename,'w',encoding='utf-8') as w:          
         for line in corp:
           w.write(line)
           w.write('\n')
################# ######Step6#############################

################ Evaluate summary  #################
 guj_remove_newlines(norm_gold_file,norm_gold_file)
 
 guj_remove_newlines(summary_filename,summary_filename)
 rougescores=evalate_files(norm_gold_file,summary_filename)#get the evaluation of generated summary after post processing based on rouge score
 for key in rougescores.keys():
     resultlist.append("%s,%s"%(key,rougescores[key]))
 resultvar.writerow(resultlist)   
 analyzedfile_obj.write("\n **********************Rouge scores based analysis for {0} % extraction is as follows*********************** ".format(percent))
 analyzedfile_obj.write("\n\n The rouge scores on summary file are \n {0}".format(rougescores,2)) #store the results 
 basicfile="original.txt"
  #guj_clean_text(filename,basicfile) # Get the original file in cleaned form for computing compression ratio. 
 guj_remove_newlines(filename,basicfile) 
  
 sent_compression,word_compression,totaloriginalsentences,totaloriginalwords,totalsumsentences,totalsumwords=compression_statistics(summary_filename,basicfile) #to store the compression ratio analysis
 analyzedfile_obj.write("\n **********************Compression Analysis at {0} % extraction is as follows(based on original file) *********************** ".format(percent))
 analyzedfile_obj.write("\n Total sentences in original text were {0} ".format(totaloriginalsentences)) 
 analyzedfile_obj.write("\n Total sentences in summary text are {0} ".format(totalsumsentences))
 analyzedfile_obj.write("\n Sentences compression ratio is {0} \n".format(sent_compression,2))
 analyzedfile_obj.write(" \n\n Total words in original text were {0} ".format(totaloriginalwords))
 analyzedfile_obj.write(" \n total words in summary text are {0} ".format(totalsumwords))
 analyzedfile_obj.write("\n Words compression ratio is {0} ".format(word_compression,2)) 

  
 data = {'filename':[filename],
          'totaloriginalsentences':[totaloriginalsentences],
          'totalsumsentences':[totalsumsentences],
           'sent_compression':[sent_compression],
           'totaloriginalwords':[totaloriginalwords],
           'totalsumwords':[totalsumwords],
          'word_compression':[word_compression]
          
          }
        
 df1=pd.DataFrame(data)
 df1.to_csv(filename_statistics,mode='a',header=False) 

{'rouge-1': {'f': 0.963184074605599, 'p': 0.9897750511247444, 'r': 0.937984496124031}, 'rouge-2': {'f': 0.957171309744652, 'p': 0.9836233367451381, 'r': 0.9321047526673133}, 'rouge-l': {'f': 0.9725906227641713, 'p': 0.9874326750448833, 'r': 0.9581881533101045}}


In [307]:
analyzedfile_obj.close()

rougescorefilehandle.close()

In [ ]:
## code to download all data at end of execution .Uncomment it after all experiments done. 
"""
from google.colab import files

!zip -r /content/txtfiles.zip /content/*.txt
files.download("txtfiles.zip")
!zip -r /content/csvfiles.zip /content/*.csv
files.download("csvfiles.zip")
!zip -r /content/codesfiles.zip /content/*.py
files.download("codesfiles.zip")
"""
